In [47]:
from fastai.vision import *
from keras.preprocessing.image import ImageDataGenerator
import tensorflow
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras import backend as k
import requests
import json
from urllib.request import urlretrieve
import csv
from keras import backend as K
from PIL import ImageFile, Image
ImageFile.LOAD_TRUNCATED_IMAGES = True
from keras.preprocessing import image

In [48]:
response = json.loads(requests.get("http://arnelism.com/realestate/labelled/files.json").text)
response_documents = json.loads(requests.get("https://raw.githubusercontent.com/perens/picture-labelling/main/src/resources/dokumendi_blanketid.json").text)

In [ ]:
!mkdir train
!mkdir train/facade0
!mkdir train/interior0
!mkdir train/document0

In [50]:
facades = [url for url in response if "facade" in url]
interiors = [url for url in response if "interior" in url]
documents = [url for url in response_documents]

In [42]:
print(len(facades))
fadaces_train = facades[0:450]
facades_valid = facades[450:500]
facades_test = facades[500:550]
print(len(interiors))
interiors_train = interiors[0:400]
interiors_valid = interiors[400:430]
interiors_test = interiors[430:470]
print(len(documents))
documents_train = documents[0:150]
documents_valid = documents[150:190]
documents_test = documents[190:220]

562
473
226


In [51]:
f = open("facades_train.csv", "a")
for url in fadaces_train:
  f.write(url + "\n")
f.close()

f = open("interiors_train.csv", "a")
for url in interiors_train:
  f.write(url + "\n")
f.close()

f = open("documents_train.csv", "a")
for url in documents_train:
  f.write(url + "\n")
f.close()

In [52]:
f = open("facades_valid.csv", "a")
for url in facades_valid:
  f.write(url + "\n")
f.close()

f = open("interiors_valid.csv", "a")
for url in interiors_valid:
  f.write(url + "\n")
f.close()

f = open("documents_valid.csv", "a")
for url in documents_valid:
  f.write(url + "\n")
f.close()

In [53]:
f = open("facades_test.csv", "a")
for url in facades_test:
  f.write(url + "\n")
f.close()

f = open("interiors_test.csv", "a")
for url in interiors_test:
  f.write(url + "\n")
f.close()

f = open("documents_test.csv", "a")
for url in documents_test:
  f.write(url + "\n")
f.close()

In [54]:

download_images("facades_train.csv", 'train/facade0', max_pics=1000) 
download_images("interiors_train.csv", 'train/interior0', max_pics=1000)
download_images("documents_train.csv", 'train/document0', max_pics=1000)

In [59]:
!mkdir validation
!mkdir validation/facade0
!mkdir validation/interior0
!mkdir validation/document0
!mkdir test
!mkdir test/facade0
!mkdir test/interior0
!mkdir test/document0

mkdir: cannot create directory ‘validation’: File exists
mkdir: cannot create directory ‘validation/facade0’: File exists
mkdir: cannot create directory ‘validation/interior0’: File exists
mkdir: cannot create directory ‘validation/document0’: File exists


In [60]:
download_images("facades_valid.csv", 'validation/facade0', max_pics=100)
download_images("interiors_valid.csv", 'validation/interior0', max_pics=100)
download_images("documents_valid.csv", 'validation/document0', max_pics=100)

In [61]:
download_images("facades_test.csv", 'test/facade0', max_pics=100)
download_images("interiors_test.csv", 'test/interior0', max_pics=100)
download_images("documents_test.csv", 'test/document0', max_pics=100)

In [62]:
def make_model():
  model = Sequential()
  model.add(Conv2D(32, (3,3), input_shape=input_shape))
  model.add(Activation('relu'))
  model.add(Conv2D(32, (3,3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))

  model.add(Conv2D(64, (3,3)))
  model.add(Activation('relu'))
  model.add(Conv2D(64, (3,3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))

  model.add(Conv2D(128, (3,3),))
  model.add(Activation('relu'))
  model.add(Conv2D(128, (3,3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))

  model.add(Dense(100, activation='relu'))

  model.add(Flatten())
  model.add(Dense(3, activation='softmax'))

  return model

In [63]:
img_width, img_height = 256, 256

train_data_dir = 'train'
validation_data_dir = 'validation'
nb_train_samples = 1000
nb_validation_samples = 160
epochs = 20
batch_size = 20

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [64]:
model = make_model()

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

train_datagen = ImageDataGenerator(rescale=1. / 255,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 2500 images belonging to 3 classes.
Found 300 images belonging to 3 classes.


In [65]:
m = model.fit(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size
    )

Epoch 1/20
50/50 [==============================] - 17s 340ms/step - loss: 0.9047 - accuracy: 0.6470 - val_loss: 0.9105 - val_accuracy: 0.7188
Epoch 2/20
50/50 [==============================] - 17s 340ms/step - loss: 0.4584 - accuracy: 0.8630 - val_loss: 0.2590 - val_accuracy: 0.9438
Epoch 3/20
50/50 [==============================] - 17s 339ms/step - loss: 0.2771 - accuracy: 0.9120 - val_loss: 0.3084 - val_accuracy: 0.9312
Epoch 4/20
50/50 [==============================] - 17s 339ms/step - loss: 0.2321 - accuracy: 0.9310 - val_loss: 0.3496 - val_accuracy: 0.9125
Epoch 5/20
50/50 [==============================] - 17s 337ms/step - loss: 0.2302 - accuracy: 0.9260 - val_loss: 0.4725 - val_accuracy: 0.8750
Epoch 6/20
50/50 [==============================] - 17s 340ms/step - loss: 0.2769 - accuracy: 0.9180 - val_loss: 0.2444 - val_accuracy: 0.9250
Epoch 7/20
50/50 [==============================] - 17s 342ms/step - loss: 0.2145 - accuracy: 0.9410 - val_loss: 0.3433 - val_accuracy: 0.8938

In [76]:
print(train_generator.class_indices)
for directory in ["test/facade0", "test/interior0", "test/document0"]:
  print(directory)
  valid = train_generator.class_indices[directory.split('/')[1]]
  counter = 0
  p = 0
  for entry in os.scandir(directory):
    if entry.path.endswith(".jpg") and entry.is_file():
      img = image.load_img(entry.path, target_size = (img_width, img_height))
      img = image.img_to_array(img)
      img = np.expand_dims(img, axis = 0)
      arvamus = np.argmax(model.predict(img), axis=1)
      if arvamus == valid:
        p += 1
      counter += 1
  print((p/counter)*100)

{'document0': 0, 'facade0': 1, 'interior0': 2}
test/facade0
96.0
test/interior0
20.0
test/document0
93.33333333333333
